In [6]:
import pandas as pd
import random
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import apply_affine_transform as Affine, apply_channel_shift as CShift
from utils import *

val_info = pd.read_csv('./val_info.csv')
info = pd.read_csv('./image_info.csv')

In [56]:


# Partitioning the data
# c1 = info.query('mask == 1 & whites < 800').sample(n=100,random_state=101).reset_index(drop=True)
# c2 = info.query('mask == 1 & whites >= 800 & whites < 1600').sample(n=100,random_state=101).reset_index(drop=True)
# c3 = info.query('mask == 1 & whites >= 1600 & whites < 2400').sample(n=100,random_state=101).reset_index(drop=True)
# c4 = info.query('mask == 1 & whites >= 2400 & whites < 3200').sample(n=100,random_state=101).reset_index(drop=True)
# c5 = info.query('mask == 1 & whites >= 3200').sample(n=100,random_state=101).reset_index(drop=True)

# c1['Category'] = 'C1'
# c2['Category'] = 'C2'
# c3['Category'] = 'C3'
# c4['Category'] = 'C4'
# c5['Category'] = 'C5'

# raw_eval = pd.concat([c1,c2,c3,c4,c5]).reset_index(drop=True)
# raw_eval.drop(['mask'], axis=1, inplace=True)

# gen_val_data = {
#     'file_name': [],
#     'img_path': [],
#     'mask_path': [],
#     'whites': [],
#     'Category' : []
# }

# def gen_val_image(image_path, mask_path, whites, cat):
    
#     name = image_path.split('/')[-1][:-4]
#     im = load_image(image_path)
#     ims = [None, None, None]

#     ims[0] = power_law_combination(im, [1.2, 1.5, 1.6])
#     ims[1] = power_law_combination(im, [0.1, 1.7, 2.3])
#     ims[2] = power_law_combination(im, [0.7, 1.1, 1.5])

#     OUTPUT_PATH = './augmentation/validation/'

#     for i in range(3):
#         file_name = f'{name}_{i}.tif'
#         im_path = f'{OUTPUT_PATH}{file_name}.tif'
#         cv2.imwrite(im_path, ims[i])

#         gen_val_data['file_name'].append(file_name)
#         gen_val_data['img_path'].append(im_path)
#         gen_val_data['mask_path'].append(mask_path)
#         gen_val_data['whites'].append(whites)
#         gen_val_data['Category'].append(cat)

# _ = raw_eval.apply(lambda x: gen_val_image(x.img_path, x.mask_path, x.whites, x.Category), axis=1)

# pd.concat([raw_eval, pd.DataFrame(gen_val_data)]).sort_values(['Category', 'whites']).reset_index(drop=True).to_csv('val_info.csv', index=0)

,file_name,img_path,mask_path,whites,mask
0,TCGA_FG_A60K_20040224_62,./archive/kaggle_3m/TCGA_FG_A60K_20040224/TCGA...,./archive/kaggle_3m/TCGA_FG_A60K_20040224/TCGA...,0,0
1,TCGA_HT_A61A_20000127_30,./archive/kaggle_3m/TCGA_DU_A5TT_19980318/TCGA...,./archive/kaggle_3m/TCGA_DU_A5TT_19980318/TCGA...,0,0
2,TCGA_HT_7877_19980917_16,./archive/kaggle_3m/TCGA_DU_6400_19830518/TCGA...,./archive/kaggle_3m/TCGA_DU_6400_19830518/TCGA...,0,0
3,TCGA_DU_6399_19830416_21,./archive/kaggle_3m/TCGA_DU_7309_19960831/TCGA...,./archive/kaggle_3m/TCGA_DU_7309_19960831/TCGA...,0,0
4,TCGA_FG_A4MT_20020212_29,./archive/kaggle_3m/TCGA_FG_8189_20030516/TCGA...,./archive/kaggle_3m/TCGA_FG_8189_20030516/TCGA...,0,0
...,...,...,...,...,...
995,TCGA_EZ_7264_20010816_20,./archive/kaggle_3m/TCGA_DU_6401_19831001/TCGA...,./archive/kaggle_3m/TCGA_DU_6401_19831001/TCGA...,0,0
996,TCGA_FG_A60K_20040224_68,./archive/kaggle_3m/TCGA_HT_7881_19981015/TCGA...,./archive/kaggle_3m/TCGA_HT_7881_19981015/TCGA...,0,0
997,TCGA_DU_5871_19941206_16,./archive/kaggle_3m/TCGA_DU_8163_19961119/TCGA...,./archive/kaggle_3m/TCGA_DU_8163_19961119/TCGA...,0,0
998,TCGA_CS_6666_20011109_7,./archive/kaggle_3m/TCGA_FG_6690_20020226/TCGA...,./archive/kaggle_3m/TCGA_FG_6690_20020226/TCGA...,0,0


In [14]:
raw = info.query('mask == 0').sample(n=1000, random_state=101).reset_index(drop=True)

gen_aug_data = {
    'file_name': [],
    'img_path': [],
    'mask_path': [],
    'whites': []
}

def gen_aug_image(image_path, mask_path, whites):
    
    name = image_path.split('/')[-1][:-4]
    im = load_image(image_path)
    im = power_law_combination(im, [0.1, 1.7, 2.3])

    OUTPUT_PATH = './augmentation/validation_overall/'
    
    file_name = f'{name}_aug.tif'
    im_path = f'{OUTPUT_PATH}{file_name}.tif'
    cv2.imwrite(im_path, im)

    gen_aug_data['file_name'].append(file_name)
    gen_aug_data['img_path'].append(im_path)
    gen_aug_data['mask_path'].append(mask_path)
    gen_aug_data['whites'].append(whites)

_ = raw.apply(lambda x: gen_aug_image(x.img_path, x.mask_path, x.whites), axis=1)

In [20]:
df = pd.concat([pd.DataFrame(gen_aug_data), raw, val_info]).reset_index(drop=True)

In [22]:
df.to_csv('overall_val.csv', index=False)

In [23]:
from keras.models import load_model
from tensorflow.keras import backend as K

smooth =100

def iou(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac

def dice_coef(y_true, y_pred):
    y_truef=K.flatten(y_true)
    y_predf=K.flatten(y_pred)
    And=K.sum(y_truef* y_predf)
    return((2* And + smooth) / (K.sum(y_truef) + K.sum(y_predf) + smooth))

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def get_batch(df):
    LEN = len(df)

    ims = []
    mss = []

    for index, row in df.iterrows():
        ims.append(load_image(row['img_path']))
        mss.append(load_image(row['mask_path']))

    return np.array(ims), np.array(mss).reshape((LEN, 256, 256, 1))

In [3]:
def eval_model(model_name):
    model = load_model(f'./models/{model_name}', compile=False)

    avg_iou = 0
    avg_dice = 0

    for index in range(1, 6):
        images, masks = get_batch(val_info.query(f"Category == 'C{index}'"))

        preds = model.predict(images)

        iou_metric  = round(iou(preds, masks).numpy(), 7)
        dice_metric = round(dice_coef(preds, masks).numpy(), 7)
        
        print(f'Category {index}')
        print(f'iou : {iou_metric}')
        print(f'dice: {dice_metric}')

        avg_iou += iou_metric
        avg_dice += dice_metric

    avg_iou /=  5
    avg_dice /= 5    
    
    print(f'Average IOU: {avg_iou}')
    print(f'Average Dice: {avg_dice}')

In [26]:
def general_eval(model_names):

    images, masks = get_batch(df)

    for model_name in model_names:
        model = load_model(f'./models/{model_name}', compile=False)

        preds = model.predict(images)

        iou_metric  = round(iou(preds, masks).numpy(), 7)
        dice_metric = round(dice_coef(preds, masks).numpy(), 7)
        print(f'Model {model_name}')
        print(f'iou : {iou_metric}')
        print(f'dice: {dice_metric}')
        print('----------------------------------')

names = [
    'base_model.hdf5',
    'cpl_unbalanced.h5',
    'cpl_balanced.h5',
    'lg_unbalanced.h5',
    'lg_balanced.h5',
    'log_cpl_unbalanced.h5',
    'log_cpl_balanced.h5'
]

general_eval(names)

2022-12-08 23:04:26.886395: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


125/125 [==============================] - 283s 2s/step
Model base_model.hdf5
iou : 3.5000000480067683e-06
dice: 0.014496900141239166
----------------------------------


2022-12-08 23:09:15.435360: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


125/125 [==============================] - 329s 3s/step
Model cpl_unbalanced.h5
iou : 0.9813082218170166
dice: 5.18000015290454e-05
----------------------------------


2022-12-08 23:14:50.115070: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

125/125 [==============================] - 384s 3s/step
Model lg_balanced.h5
iou : 0.013797200284898281
dice: 0.0002610000083222985
----------------------------------


2022-12-08 23:33:39.635127: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


125/125 [==============================] - 398s 3s/step
Model log_cpl_unbalanced.h5
iou : 0.9963842034339905
dice: 5.1700000767596066e-05
----------------------------------


2022-12-08 23:40:23.363490: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


125/125 [==============================] - 422s 3s/step
Model log_cpl_balanced.h5
iou : 0.9398316740989685
dice: 5.1700000767596066e-05
----------------------------------


In [98]:
for index in range(1, 6):
    images, masks = get_batch(val_info.query(f"Category == 'C{index}'"))

    preds = base.predict(images)
    
    print(f'Category {index}')
    print(f'iou : {round(iou(preds, masks).numpy(), 7)}')
    print(f'dice: {round(dice_coef(preds, masks).numpy(), 7)}')

13/13 [==============================] - 27s 2s/step
Category 1
iou : 0.0033605999778956175
dice: 0.006690099835395813
13/13 [==============================] - 26s 2s/step
Category 2
iou : 0.009964300319552422
dice: 0.0197237990796566
13/13 [==============================] - 27s 2s/step
Category 3
iou : 0.015772299841046333
dice: 0.031046800315380096
13/13 [==============================] - 34s 3s/step
Category 4
iou : 0.024824099615216255
dice: 0.04843800142407417
13/13 [==============================] - 36s 3s/step
Category 5
iou : 0.07278910279273987
dice: 0.1356942057609558


In [111]:
eval_model('base_model.hdf5')

2022-12-07 19:06:14.727484: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 29s 2s/step
Category 1
iou : 0.0033605999778956175
dice: 0.006690099835395813
13/13 [==============================] - 29s 2s/step
Category 2
iou : 0.009964300319552422
dice: 0.0197237990796566
13/13 [==============================] - 29s 2s/step
Category 3
iou : 0.015772299841046333
dice: 0.031046800315380096
13/13 [==============================] - 29s 2s/step
Category 4
iou : 0.024824099615216255
dice: 0.04843800142407417
13/13 [==============================] - 42s 3s/step
Category 5
iou : 0.07278910279273987
dice: 0.1356942057609558
Average IOU: 0.0253420805092901
Average Dice: 0.0483185812830925


In [112]:
eval_model('cpl_unbalanced.h5')

2022-12-07 19:13:03.509131: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 29s 2s/step
Category 1
iou : 0.056507598608732224
dice: 0.10677459836006165
13/13 [==============================] - 28s 2s/step
Category 2
iou : 0.15511339902877808
dice: 0.2684659957885742
13/13 [==============================] - 33s 3s/step
Category 3
iou : 0.30292779207229614
dice: 0.4649254083633423
13/13 [==============================] - 43s 3s/step
Category 4
iou : 0.35651201009750366
dice: 0.5255857706069946
13/13 [==============================] - 45s 4s/step
Category 5
iou : 0.6050001978874207
dice: 0.7538836002349854
Average IOU: 0.2952121995389462
Average Dice: 0.4239270746707916


In [113]:
eval_model('cpl_balanced.h5')

2022-12-07 19:16:11.306394: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 43s 3s/step
Category 1
iou : 0.07132740318775177
dice: 0.13297079503536224
13/13 [==============================] - 43s 3s/step
Category 2
iou : 0.22748729586601257
dice: 0.3705432116985321
13/13 [==============================] - 42s 3s/step
Category 3
iou : 0.33525100350379944
dice: 0.5020859837532043
13/13 [==============================] - 44s 3s/step
Category 4
iou : 0.4557693898677826
dice: 0.626114010810852
13/13 [==============================] - 46s 4s/step
Category 5
iou : 0.7261579036712646
dice: 0.8413501977920532
Average IOU: 0.3631985992193222
Average Dice: 0.4946128398180008


In [114]:
eval_model('lg_balanced.h5')

2022-12-07 19:22:39.280231: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 28s 2s/step
Category 1
iou : 0.14958590269088745
dice: 0.25998660922050476
13/13 [==============================] - 26s 2s/step
Category 2
iou : 0.343948096036911
dice: 0.5117455720901489
13/13 [==============================] - 27s 2s/step
Category 3
iou : 0.4663054049015045
dice: 0.635971188545227
13/13 [==============================] - 27s 2s/step
Category 4
iou : 0.5754625797271729
dice: 0.7305004000663757
13/13 [==============================] - 29s 2s/step
Category 5
iou : 0.7967522144317627
dice: 0.8868753910064697
Average IOU: 0.4664108395576477
Average Dice: 0.6050158321857453


In [115]:
eval_model('lg_unbalanced.h5')

2022-12-07 19:25:03.884269: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 31s 2s/step
Category 1
iou : 0.26744669675827026
dice: 0.42152079939842224
13/13 [==============================] - 35s 3s/step
Category 2
iou : 0.43174439668655396
dice: 0.6029621958732605
13/13 [==============================] - 72s 6s/step
Category 3
iou : 0.5403130054473877
dice: 0.7014977931976318
13/13 [==============================] - 54s 4s/step
Category 4
iou : 0.623761773109436
dice: 0.7682579159736633
13/13 [==============================] - 47s 4s/step
Category 5
iou : 0.7528527975082397
dice: 0.8589956760406494
Average IOU: 0.5232237339019775
Average Dice: 0.6706468760967255


In [116]:
eval_model('log_cpl_unbalanced.h5')

2022-12-07 19:29:13.012500: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 46s 4s/step
Category 1
iou : 0.03112870082259178
dice: 0.06030679866671562
13/13 [==============================] - 46s 4s/step
Category 2
iou : 0.08417230099439621
dice: 0.15522019565105438
13/13 [==============================] - 44s 3s/step
Category 3
iou : 0.12287890166044235
dice: 0.2188224047422409
13/13 [==============================] - 47s 4s/step
Category 4
iou : 0.21298609673976898
dice: 0.3511398136615753
13/13 [==============================] - 48s 4s/step
Category 5
iou : 0.45486781001091003
dice: 0.6252923011779785
Average IOU: 0.18120676204562186
Average Dice: 0.28215630277991294


In [4]:
eval_model('log_cpl_balanced.h5')

2022-12-08 10:41:28.521915: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-08 10:41:28.628926: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 29s 2s/step
Category 1
iou : 0.021290499716997147
dice: 0.04164940118789673
13/13 [==============================] - 27s 2s/step
Category 2
iou : 0.04979820176959038
dice: 0.09483770281076431
13/13 [==============================] - 29s 2s/step
Category 3
iou : 0.07712479680776596
dice: 0.1431758999824524
13/13 [==============================] - 28s 2s/step
Category 4
iou : 0.11306949704885483
dice: 0.20313960313796997
13/13 [==============================] - 26s 2s/step
Category 5
iou : 0.2569184899330139
dice: 0.40879279375076294
Average IOU: 0.10364029705524444
Average Dice: 0.17831908017396927
